# Fixer les inchérence entre SIRET et SIREN des Acteurs affichés

1. 

In [111]:
from pathlib import Path
import decouple

# Notebook : data-platform/notebooks
notebooks_dir = Path.cwd()
project_root = notebooks_dir.parent        # => data-platform
repo_root = project_root.parent            # => racine du repo
webapp_dir = repo_root / "webapp"

# On force decouple à lire webapp/.env
decouple.config = decouple.AutoConfig(search_path=str(webapp_dir))



In [112]:
import sys
import os
from pathlib import Path

# On part du dossier du notebook: data-platform/notebooks
project_root = Path.cwd().parent   # => data-platform
sys.path.append(str(project_root))
sys.path.append(str(project_root / "dags"))

from dags.utils.django import django_setup_full

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django_setup_full()

In [113]:
from django.db import connections

# Fermer explicitement les connexions existantes
for conn in connections.all():
    conn.close()


In [114]:
from django.db.models import Q
from django.db.models.functions import Length

from qfdmo.models.acteur import VueActeur

acteurs = VueActeur.objects.filter(Q(est_dans_opendata=True) | Q(est_dans_carte=True))

print(acteurs.count())


384248


In [115]:
acteurs_siret_9_no_siren = (
    acteurs
    .annotate(siret_length=Length('siret'))
    .filter(siret_length=9, siren='')
    .exclude(siret='__empty__')
)

print(acteurs_siret_9_no_siren.count())

1731


In [116]:
for acteur in acteurs_siret_9_no_siren:
    #check if siren existe in warehouse.clone_ae_unite_legale_in_use and is actif
    from django.db import connections
    with connections['warehouse'].cursor() as cursor:
        cursor.execute("SELECT siren, nom FROM clone_ae_unite_legale_in_use WHERE siren = %s and etat_administratif = 'A'", [acteur.siret])
        result = cursor.fetchone()
        if result:
            print(f"SIREN found and actif: {result}")
            # TODO: update acteur.siren with the siren found

            # Search if only 1 siret Actif for the given siren
            cursor.execute("SELECT siren,siret,code_postal FROM clone_ae_etablissement_in_use WHERE siren = %s and etat_administratif = 'A' and code_postal = %s", [acteur.siret, acteur.code_postal])
            # count the number of result
            result = cursor.fetchall()
            if len(result) == 1:
                print(f"Only 1 siret Actif for the given siren: {result}")
                # TODO: update acteur.siret with the siret found
            else:
                print(f"More than 1 siret Actif for the given siren: {result}")
        else:
            print(f"SIREN not found or not actif: {acteur.siret}")


SIREN found and actif: ('453798530', None)
Only 1 siret Actif for the given siren: [('453798530', '45379853000011', '44300')]
SIREN found and actif: ('424753820', 'GARCIA')
More than 1 siret Actif for the given siren: [('424753820', '42475382000036', '31270'), ('424753820', '42475382000044', '31270')]
SIREN found and actif: ('200034700', None)
More than 1 siret Actif for the given siren: []
SIREN found and actif: ('912734936', None)
Only 1 siret Actif for the given siren: [('912734936', '91273493600020', '45480')]
SIREN found and actif: ('921930525', None)
Only 1 siret Actif for the given siren: [('921930525', '92193052500024', '38280')]
SIREN found and actif: ('751698770', None)
Only 1 siret Actif for the given siren: [('751698770', '75169877000045', '44700')]
SIREN found and actif: ('746950179', None)
More than 1 siret Actif for the given siren: []
SIREN found and actif: ('402432439', None)
Only 1 siret Actif for the given siren: [('402432439', '40243243900089', '76000')]
SIREN found